In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from scipy import sparse
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

In [4]:
df = pd.read_csv("diabetic_data.csv")
ids_mapping = pd.read_csv("IDS_mapping.csv")

Preprocessing

In [5]:
# Predict whether or not hospital patient will be readmitted within 30 days

# Replace "?" with NaN to standardize missing values
df = df.replace("?", np.nan)

# Target variable includes those recently readmitted (within 30 days)
# creates a binary target where 1 = readmitted <30, 0 otherise
if "readmitted" in df.columns:
    df["target"] = df["readmitted"].apply(lambda x: 1 if x == "<30" else 0)
    df = df.drop(["readmitted"], axis=1)
    
# Drop ID and unhelpful columns
drop_cols = ["encounter_id", "patient_nbr", "weight", "payer_code", "medical_specialty", "examide",  "citoglipton", "metformin-rosiglitazone", "metformin-pioglitazone"]
df = df.drop(columns=drop_cols, errors='ignore')

# Drop columns with excessive missing values or high cardinality
df = df.loc[:, df.isnull().mean() < 0.5]
df = df.drop(columns=[col for col in df.select_dtypes(include="object") if df[col].nunique() > 50])
df = df.dropna()

# One-hot encode with float dtype to avoid object dtype issues
df_dummies = pd.get_dummies(df, drop_first=True, dtype=float)

# Separate features and target
X = df_dummies.drop("target", axis=1)
y = df_dummies["target"]

# Convert to scipy sparse matrix
X_sparse = sparse.csr_matrix(X.values)

# Train/test split (sparse matrix version)
X_train_sparse, X_test_sparse, y_train, y_test = train_test_split(
    X_sparse, y, test_size=0.3, random_state=42, stratify=y
)


# Feature scaling using sparse-compatible scaler (no centering)
scaler = StandardScaler(with_mean=False)
X_train_scaled = scaler.fit_transform(X_train_sparse)
X_test_scaled = scaler.transform(X_test_sparse)

# Prepare to store metrics for later model evaluation
metrics = {}

In [ ]:
'''
    decided to use t-sne because it allows us to turn this multi dimensional space into a 2d space
    therefore, we can see what model is the best to use, as a guess  
    then we can compare THIS inital guess to the FINAL result
'''
tsne = TSNE(n_components=2, random_state=42, perplexity=30, max_iter=1000)
X_embedded = tsne.fit_transform(X_train_scaled.toarray())

# Plot
plt.figure(figsize=(10, 6))
# 0 (NOT readmitted) = blue || 1 (readmitted) = red
scatter = plt.scatter(X_embedded[:, 0], X_embedded[:, 1], c=y_train, cmap="coolwarm", alpha=0.6)
plt.colorbar(scatter, label="Readmitted (<30 days = 1)")
plt.grid(True)
plt.show()

k-Nearest Neighors

In [7]:
# Convert sparse matrices to dense arrays (KNN doesn't support sparse input)
X_train_dense = X_train_scaled.toarray()
X_test_dense = X_test_scaled.toarray()

metrics["kNN"] = {}

for k in range(1, 21):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_dense, y_train)
    
    y_pred_knn = knn.predict(X_test_dense)
    
    metrics["kNN"][k] = {
        "Accuracy": accuracy_score(y_test, y_pred_knn),
        "Precision": precision_score(y_test, y_pred_knn, zero_division=0),
        "Recall": recall_score(y_test, y_pred_knn),
        "F1": f1_score(y_test, y_pred_knn),
        "AUC": roc_auc_score(y_test, y_pred_knn)
    }

    print(f"\nk = {k}")
    for metric_name, value in metrics["kNN"][k].items():
        print(f"{metric_name}: {value:.4f}")

# Extract metric values for each k
ks = list(metrics["kNN"].keys())
accuracy = [metrics["kNN"][k]["Accuracy"] for k in ks]
precision = [metrics["kNN"][k]["Precision"] for k in ks]
recall = [metrics["kNN"][k]["Recall"] for k in ks]
f1 = [metrics["kNN"][k]["F1"] for k in ks]
auc = [metrics["kNN"][k]["AUC"] for k in ks]

plt.figure(figsize=(10, 6))

plt.plot(ks, accuracy, label="Accuracy", marker='o')
plt.plot(ks, precision, label="Precision", marker='o')
plt.plot(ks, recall, label="Recall", marker='o')
plt.plot(ks, f1, label="F1 Score", marker='o')
plt.plot(ks, auc, label="AUC", marker='o')

plt.xlabel("Number of Neighbors (k)")
plt.ylabel("Score")
plt.title("KNN Classifier Metrics vs k")
plt.xticks(ks)
plt.legend()
plt.grid(True)
plt.show()

Accuracy: 0.8791
Precision: 0.2480
Recall: 0.0379
F1: 0.0658
AUC: 0.5117


Logistic Regression

In [13]:
# Define a list of candidate values for C (inverse of regularization strength)
candidate_C_values = [0.01, 0.1, 1, 10, 100]

best_score = 0
best_C = None

for C in candidate_C_values:
    # Initialize logistic regression with current C and other fixed hyperparameters
    model = LogisticRegression(
        penalty='l2',
        class_weight='balanced',
        max_iter=1000,
        random_state=42,
        solver='lbfgs',
        C=C
    )
    
    # Perform k-fold cross-validation on training data
    scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='roc_auc')
    
    # Calculate mean validation score
    mean_score = scores.mean()
    
    # Update best parameters if current mean score is better
    if mean_score > best_score:
        best_score = mean_score
        best_C = C

# Train final model using best_C on full training data
final_model = LogisticRegression(
    penalty='l2', 
    class_weight='balanced', 
    max_iter=1000, 
    random_state=42, 
    solver='lbfgs', 
    C=best_C
)
final_model.fit(X_train_scaled, y_train)
y_pred_lr = lr.predict(X_test_scaled)

metrics["Logistic Regression"] = {
    "Accuracy": accuracy_score(y_test, y_pred_lr),
    "Precision": precision_score(y_test, y_pred_lr),
    "Recall": recall_score(y_test, y_pred_lr),
    "F1": f1_score(y_test, y_pred_lr),
    "AUC": roc_auc_score(y_test, y_pred_lr)
}

for metric_name, value in metrics["Logistic Regression"].items():
    print(f"{metric_name}: {value:.4f}")

Accuracy: 0.6669
Precision: 0.1717
Recall: 0.5142
F1: 0.2574
AUC: 0.6002


Feedforward Neural Network

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=1000, random_state=42)
mlp.fit(X_train_scaled, y_train)
y_pred_mlp = mlp.predict(X_test_scaled)

metrics["Feedforward NN"] = {
    "Accuracy": accuracy_score(y_test, y_pred_mlp),
    "Precision": precision_score(y_test, y_pred_mlp),
    "Recall": recall_score(y_test, y_pred_mlp),
    "F1": f1_score(y_test, y_pred_mlp),
    "AUC": roc_auc_score(y_test, y_pred_mlp)
}
for metric_name, value in metrics["Feedforward NN"].items():
    print(f"{metric_name}: {value:.4f}")

Decision Tree

In [ ]:
metrics["Decision Tree"] = {}

for depth in range(31,60):
    dt = DecisionTreeClassifier(max_depth=depth, random_state=42)
    dt.fit(X_train_scaled, y_train)
    y_pred_dt = dt.predict(X_test_scaled)
    
    metrics["Decision Tree"][depth] = {
        "Accuracy": accuracy_score(y_test, y_pred_dt),
        "Precision": precision_score(y_test, y_pred_dt, zero_division=0),
        "Recall": recall_score(y_test, y_pred_dt),
        "F1": f1_score(y_test, y_pred_dt),
        "AUC": roc_auc_score(y_test, y_pred_dt)
    }
    print(f"\nmax_depth = {depth}")
    for metric_name, value in metrics["Decision Tree"][depth].items():
        print(f"{metric_name}: {value:.4f}")


max_depth = 31
Accuracy: 0.8002
Precision: 0.1514
Recall: 0.1692
F1: 0.1598
AUC: 0.5246

max_depth = 32
Accuracy: 0.8001
Precision: 0.1592
Recall: 0.1823
F1: 0.1700
AUC: 0.5303

max_depth = 33
Accuracy: 0.7974
Precision: 0.1538
Recall: 0.1788
F1: 0.1653
AUC: 0.5272


Confusion Matrices

In [ ]:
def plot_confusion(title, y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap="Blues")
    plt.title(title)
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()

plot_confusion("kNN Confusion Matrix", y_test, y_pred_knn)
plot_confusion("Logistic Regression Confusion Matrix", y_test, y_pred_lr)
plot_confusion("Feedforward NN Confusion Matrix", y_test, y_pred_mlp)
plot_confusion("Decision Tree Confusion Matrix", y_test, y_pred_dt)

Visualization of Metrics

In [ ]:
def plot_metrics(metrics_dict):
    metrics_names = ["Accuracy", "Precision", "Recall", "F1", "AUC"]
    classifiers = list(metrics_dict.keys())
    values = {metric: [metrics_dict[classifier][metric] for classifier in classifiers] for metric in metrics_names}

    x = np.arange(len(classifiers))
    width = 0.2

    plt.figure(figsize=(10,6))
    for i, metric in enumerate(metrics_names):
        plt.bar(x + i*width, values[metric], width=width, label=metric)

    plt.xticks(x + width*1.5, classifiers)
    plt.ylabel("Score")
    plt.title("Metrics by Classifier")
    plt.ylim(0, 1)
    plt.legend()
    plt.tight_layout()
    plt.show()

plot_metrics(metrics)